In [2]:
import pandas as pd
from matplotlib import pyplot as plt

import seaborn as sns
from sklearn.metrics import accuracy_score,precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
import chardet



C:\Users\puvit\AppData\Local\Temp\ipykernel_21888\610955669.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:

import pandas as pd
#Upload the path to the file which we have downloaded as the data is large it coundn't be uploaded
path = 'fraudTrian.csv'
dfTrain = pd.read_csv(path)

In [35]:
dfTrain.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0.0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",3/9/1988,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1.0,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2.0,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3.0,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1/12/1967,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4.0,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,3/28/1986,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [52]:
!pip install gdown

In [36]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             2002 non-null   float64
 1   trans_date_trans_time  2002 non-null   object 
 2   cc_num                 2002 non-null   float64
 3   merchant               2002 non-null   object 
 4   category               2002 non-null   object 
 5   amt                    2002 non-null   float64
 6   first                  2002 non-null   object 
 7   last                   2002 non-null   object 
 8   gender                 2002 non-null   object 
 9   street                 2002 non-null   object 
 10  city                   2002 non-null   object 
 11  state                  2002 non-null   object 
 12  zip                    2002 non-null   float64
 13  lat                    2002 non-null   float64
 14  long                   2002 non-null   float64
 15

In [37]:
def unWanted(df):
  df.drop(columns=['Unnamed: 0','trans_date_trans_time','cc_num','trans_num','state','city'], inplace=True)
  return df

In [31]:
dfTrain.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0.0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",3/9/1988,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1.0,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2.0,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3.0,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1/12/1967,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4.0,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,3/28/1986,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [ ]:
#preprocess
def trasform(df):
  le = LabelEncoder()
  for feature in ['merchant', 'category', 'first', 'last', 'gender', 'street', 'job','dob']:
      df[feature] = le.fit_transform(df[feature].astype(str))
  return df

Ytrain = dfTrain['is_fraud']
Xtrain = trasform(unWanted(dfTrain.drop(columns=['is_fraud'])))
Xtrain , Xval , Ytrain , Yval = train_test_split(Xtrain,Ytrain,test_size=0.2)

In [41]:
Xtrain.head()


,merchant,category,amt,first,last,gender,street,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long
793451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
828281,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87972,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1037206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
673570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
#train a model 
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(Xtrain, Ytrain)

RandomForestClassifier(random_state=42)

In [10]:
#Evaluvation
prediction = model.predict(Xval)
print(accuracy_score(Yval,prediction))
print(precision_score(Yval,prediction))

0.9977056702720419
0.8883025505716798


In [11]:
dfTest.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [12]:
dfTest = trasform(unWanted(dfTest))

In [13]:

dfYtest = dfTest['is_fraud']
dfTest = dfTest.drop(columns=['is_fraud'])
model.predict(dfTest)
print(accuracy_score(dfYtest,model.predict(dfTest)))
print(precision_score(dfYtest,model.predict(dfTest)))

0.9976067041076515
0.7722110888443554


#There You can predict the result :
Import the features

Pass it to unwanted function to remove Unwanted coloumn

Pass it to Trasform to apply LabelEncoder

Then call predict method

In [1]:
!pip install joblib

In [ ]:
model = joblib.load('random_forest_model (1).joblib')
model.predict()